In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# 1. Load Simulation Data
data = pd.read_csv('data/support_vector_simulation.csv')
X = data[['x1', 'x2']].values
y = data['y'].values
m, n = X.shape

# 2. Dual Objective Function for SVM
# [cite_start]Objective: Maximize the dual Lagrangian [cite: 7, 122-124]
def dual_objective(alpha, X, y):
    # We use negative for minimization
    term1 = np.sum(alpha)
    # [cite_start]Vectorized computation of the double sum [cite: 7, 140]
    K = np.dot(X, X.T)
    term2 = 0.5 * np.sum(np.outer(y, y) * np.outer(alpha, alpha) * K)
    return term2 - term1

# 3. Constraints for the Optimizer
# [cite_start]alpha_i >= 0 and sum(alpha_i * y_i) = 0 [cite: 7, 142]
constraints = ({'type': 'eq', 'fun': lambda a: np.dot(a, y)})
bounds = [(0, None) for _ in range(m)]

# 4. Optimization (Quadratic Programming)
initial_alpha = np.zeros(m)
res = minimize(fun=dual_objective, x0=initial_alpha, args=(X, y), 
               method='SLSQP', bounds=bounds, constraints=constraints)

alphas = res.x
alphas[alphas < 1e-5] = 0 # Numerical cleanup

# [cite_start]5. Recovering Parameters w and b [cite: 7, 145-147]
w = np.sum((alphas * y)[:, None] * X, axis=0)

# [cite_start]Calculate b using a support vector (alpha > 0) [cite: 7, 152]
sv_idx = np.where(alphas > 0)[0][0]
b = y[sv_idx] - np.dot(w, X[sv_idx])

print(f"Weights (w): {w}")
print(f"Intercept (b): {b}")
print(f"Support Vector Alphas: {alphas[alphas > 0]}")